In [0]:
from pyspark.sql.types import DateType,StructField,StructType,TimestampType
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.getOrCreate()
schema = StructType(
    [
        StructField("min_date", DateType()),
        StructField("end_date", TimestampType())
    ]
)

df = spark.createDataFrame(
    [("2025-12-06", "2025-12-08 07:17:57.37546")], 
    ["min_date", "end_date"]
).withColumn("min_date", col("min_date").cast("date")) \
 .withColumn("end_date", col("end_date").cast("timestamp"))


In [0]:
from pyspark.sql import functions as F
display(df)
date_seq = df.select(
    F.sequence(
        F.to_date(F.col("min_date")),
        F.to_date(F.col("end_date")),
        F.expr("interval 1 DAY")
    ).alias("cal_date")
)

date_seq = date_seq.select(F.explode("cal_date"))


display(date_seq)

In [0]:
from pyspark.sql import functions as F
F.col("min_date").between(
    F.date_add(F.col("end_date"),-6), F.col("end_date")
)